In [1]:
import torch

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:4]

['emma', 'olivia', 'ava', 'isabella']

In [3]:
for word in words[:3]:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        print(ch1, ch2, ch3)

. e m
e m m
m m a
m a .
. o l
o l i
l i v
i v i
v i a
i a .
. a v
a v a
v a .


## Exercise 1: Trigram

In [4]:
characters = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(characters)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
stoi, itos

({'a': 1,
  'b': 2,
  'c': 3,
  'd': 4,
  'e': 5,
  'f': 6,
  'g': 7,
  'h': 8,
  'i': 9,
  'j': 10,
  'k': 11,
  'l': 12,
  'm': 13,
  'n': 14,
  'o': 15,
  'p': 16,
  'q': 17,
  'r': 18,
  's': 19,
  't': 20,
  'u': 21,
  'v': 22,
  'w': 23,
  'x': 24,
  'y': 25,
  'z': 26,
  '.': 0},
 {1: 'a',
  2: 'b',
  3: 'c',
  4: 'd',
  5: 'e',
  6: 'f',
  7: 'g',
  8: 'h',
  9: 'i',
  10: 'j',
  11: 'k',
  12: 'l',
  13: 'm',
  14: 'n',
  15: 'o',
  16: 'p',
  17: 'q',
  18: 'r',
  19: 's',
  20: 't',
  21: 'u',
  22: 'v',
  23: 'w',
  24: 'x',
  25: 'y',
  26: 'z',
  0: '.'})

In [5]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)

In [6]:
for word in words:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        N[idx1, idx2, idx3] += 1
N

tensor([[[  0,   0,   0,  ...,   0,   0,   0],
         [  0, 207, 190,  ...,  27, 173, 152],
         [  0, 169,   0,  ...,   0,   4,   0],
         ...,
         [  0,  57,   0,  ...,   1,  17,  11],
         [  0, 246,   0,  ...,   0,   0,   2],
         [  0, 456,   0,  ...,   0,  91,   1]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 40,   0,   5,  ...,   0,  20,  11],
         [ 36,  28,  20,  ...,   0,  12,   0],
         ...,
         [ 11,   5,   0,  ...,  17,   6,   3],
         [163, 389,  13,  ...,   0,  16,  40],
         [ 38, 123,   0,  ...,   0,  12,  22]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 46,   5,   5,  ...,   4,  31,   4],
         [  1,   8,   0,  ...,   0,   9,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [ 55,   4,   1,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 10,   0,   2,  ...,   0,  10,   0]

In [7]:
g = torch.Generator().manual_seed(42)
P = (N+1).float()
P /= P.sum(2, keepdim=True)
P[0, 0]

tensor([0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370])

In [8]:
ex_idx = torch.multinomial(P[1, 1], 4, generator=g)
selection = [itos[i.item()] for i in ex_idx]
selection

['y', 'l', '.', 'm']

In [9]:

for i in range(10):
    out = []
    ix1 = 0
    ix2 = torch.randint(0, 27, (1,), generator=g).item()
    out.extend([ix1, ix2])
    while True:
        index = torch.multinomial(
            P[ix1, ix2], num_samples=1, replacement=True, generator=g).item()
        out.append(index)
        if index == 0:
            break
        ix1 = ix2
        ix2 = index
    # print(out)
    name = [itos[i] for i in out]
    print(''.join(name))

.wesmahle.
.man.
.roxgkthmang.
.kyn.
.layahd.
.neen.
.ana.
.na.
.brii.
.yah.


In [10]:
# computiing the overall likelihood of the dataset
log_likelihood = 0.0
n = 0

for word in words:
    chs = "." + word+"."
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        idx1, idx2, idx3 = stoi[ch1], stoi[ch2], stoi[ch3]
        log_prob = torch.log(P[idx1, idx2, idx3])
        log_likelihood += log_prob
        n += 1

print(log_likelihood)
print(n)
print(-log_likelihood/n)

tensor(-410414.9688)
196113
tensor(2.0927)


### neuralnet

In [11]:
# exemple of probabilities of next letters for the character 0(.) and 1(a)
P[0, 1]

tensor([0.0002, 0.0469, 0.0430, 0.0072, 0.0827, 0.0126, 0.0050, 0.0041, 0.0207,
        0.0349, 0.0063, 0.0171, 0.1427, 0.0868, 0.1406, 0.0025, 0.0041, 0.0023,
        0.1089, 0.0439, 0.0165, 0.0345, 0.0550, 0.0016, 0.0063, 0.0392, 0.0345])

In [34]:
xs, ys = [], []

for word in words:
    chs = '.' + word + '.'
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[3:]):
        idx1, idx2, idx3 = stoi[ch1], stoi[ch2], stoi[ch3]
        xs.append([idx1, idx2])
        ys.append(idx3)


xs, ys = torch.tensor(xs), torch.tensor(ys)
# num = ys.shape[0]
W = torch.rand((27*2, 27), generator=g, requires_grad=True)
xs.shape, ys.shape

(torch.Size([164080, 2]), torch.Size([164080]))

In [35]:
from torch.nn import functional as F
n = 1000
for i in range(n+1):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc.view(-1, 2*27) @ W
    counts = torch.exp(logits)
    # print(counts.shape)
    probs = counts/counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() - 0.1 * W.mean()

    if i % 10 == 0 or i == 0 or i == n:
        print(f"step : {i}  loss : {loss.item()}")

    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

step : 0  loss : 3.3274776935577393
step : 10  loss : 2.504183769226074
step : 20  loss : 2.4609193801879883
step : 30  loss : 2.4417195320129395
step : 40  loss : 2.429647445678711
step : 50  loss : 2.4207324981689453
step : 60  loss : 2.413508415222168
step : 70  loss : 2.407299757003784
step : 80  loss : 2.401749610900879
step : 90  loss : 2.3966519832611084
step : 100  loss : 2.391878843307495
step : 110  loss : 2.387345552444458
step : 120  loss : 2.3829946517944336
step : 130  loss : 2.3787851333618164
step : 140  loss : 2.3746883869171143
step : 150  loss : 2.3706822395324707
step : 160  loss : 2.3667492866516113
step : 170  loss : 2.3628768920898438
step : 180  loss : 2.3590548038482666
step : 190  loss : 2.3552756309509277
step : 200  loss : 2.351532459259033
step : 210  loss : 2.3478200435638428
step : 220  loss : 2.3441343307495117
step : 230  loss : 2.3404717445373535
step : 240  loss : 2.336829423904419
step : 250  loss : 2.333204746246338
step : 260  loss : 2.329595088958

In [43]:
for i in range(5):
    ix0 = 0
    ix1 = torch.randint(high=27, size=(1, 1), generator=g).item()
    out = [ix0, ix1]
    while True:
        xenc = F.one_hot(torch.tensor(
            [out[ix0], out[ix0+1]]), num_classes=27).float()
        logits = xenc.view(-1, 2*27)@W
        counts = logits.exp()
        probs = counts/counts.sum(dim=1, keepdim=True)
        ix = torch.multinomial(probs, num_samples=1,
                               replacement=True, generator=g).item()
        out.append(ix)
        if (ix == 0):
            break
        ix0 += 1
    # print(itos(out))
    print(''.join([itos[s] for s in out]))

.tli.
.vtn.
..yaansldn.
.kil.
.saiaadhn.


## Exercise 2: Train, Test,

In [ ]:
import random
random.shuffle(words)
num = len(words)
train, dev, test = words[:int(
    0.8*num)], words[int(0.8*num):int(0.9*num)], words[int(0.9*num):]
len(train) + len(dev) + len(test) == num

True

In [ ]:
for word in train:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        N[idx1, idx2, idx3] += 1

In [ ]:
xs, ys = [], []

for word in train:
    chs = '.' + word + '.'
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[3:]):
        idx1, idx2, idx3 = stoi[ch1], stoi[ch2], stoi[ch3]
        xs.append([idx1, idx2])
        ys.append(idx3)


xs, ys = torch.tensor(xs), torch.tensor(ys)
# num = ys.shape[0]
W = torch.rand((27*2, 27), generator=g, requires_grad=True)
xs.shape, ys.shape

In [ ]:
n = 10000
for i in range(n+1):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc.view(-1, 2*27) @ W
    counts = torch.exp(logits)
    # print(counts.shape)
    probs = counts/counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() - 0.1 * W.mean()

    if i % 100 == 0 or i == 0 or i == n:
        print(f"step : {i}  loss : {loss.item()}")

    W.grad = None
    loss.backward()

    W.data += -50 * W.grad